GETTING THE DATA

In [1]:
import sqlalchemy
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.join(".."))
from Src import Tools
from Src import Draw
from Src import Stats

CONNECT TO DATABASE

In [2]:
db = Tools.connect_to_dwh()

READ QUERY

In [3]:
ve = Tools.read_query("QUERY")

COMMIT QUERY

In [4]:
tnx = Tools.commit_query(ve, db)

QUERY SUMMARY

In [5]:
tnx.head()

,Customers,Transaction_Id,Sales_Amount,Date_Occurred
0,222f5fcf-8e71-47bb-9f7e-357b3642f94e,POS0007:875032845,3.95,2018-07-25
1,fcac1abc-6e0b-4649-9a50-35681a05fe30,POS0007:137504074,3.95,2018-07-27
2,d308eae3-0999-43fd-b5ea-357c6a165877,POS0007:334978027,4.00,2018-07-25
3,0a6c6337-2803-40cc-8810-356b19b6e4e0,POS0005:204907544,7.95,2018-07-23
4,6a8c5ca1-35b9-411c-b18f-356adc33015e,POS0009:413400729,3.95,2018-04-26


In [6]:
#exploring the unique values of each attribute
print("Number of rows (not empty): ", tnx.count())
print("Number of transactions (Guest Count):", tnx['Transaction_Id'].nunique())
print("Number of customers:", tnx['Customers'].nunique())

Number of rows (not empty):  Customers         128825
Transaction_Id    128825
Sales_Amount      128825
Date_Occurred     128825
dtype: int64
Number of transactions (Guest Count): 128813
Number of customers: 28520


AGGREGATE DATA

In [70]:
tnx['Lastpurchasedate']=tnx.Date_Occurred
tnx['Transaction']=tnx.Transaction_Id

In [89]:
Lastpurchasedate= tnx.groupby(["Customers"]).agg({'Date_Occurred': [np.max]})
Lastpurchasedate['DaysSinceLastPurchase'] = pd.Timestamp.now() - pd.to_datetime(Lastpurchasedate['Date_Occurred']['amax'])
Lastpurchasedate['DaysSinceLastPurchase'] = Lastpurchasedate.DaysSinceLastPurchase.dt.days
Lastpurchasedate.columns = ['LastPurchaseDate','DaysSinceLastPurchase']
Lastpurchasedate=Lastpurchasedate.reset_index()

In [72]:
Transaction_count= tnx.groupby(["Customers"]).agg({'Transaction':'nunique'})

In [77]:
TransactionAmount = tnx.groupby(["Customers"]).agg({'Sales_Amount':'sum'})
Avg_Transaction = pd.merge(TransactionAmount, Transaction_count,
                     how='left', on=['Customers'])
Avg_Transaction['Avg_Transaction_Value'] = Avg_Transaction.Sales_Amount/Avg_Transaction.Transaction

In [91]:
all_tranz = pd.merge(Avg_Transaction,Lastpurchasedate.loc[:, ['Customers', 'DaysSinceLastPurchase']],
                     how='left', on=['Customers'])

In [92]:
all_tranz.head()

,Customers,Sales_Amount,Transaction,Avg_Transaction_Value,DaysSinceLastPurchase
0,000644df-aa9b-4880-b551-357288f59223,17.7,2,8.85,50
1,000668dc-83c7-4a1f-92bb-352709cb8294,1389.0,20,69.45,7
2,0006a7d2-246c-4533-97b0-356539d30012,415.2,8,51.90,17
3,000ab955-f4b1-4c74-89bc-3569ecd81562,23.4,3,7.80,4
4,000fe1a3-a44c-4e39-8a70-352b9779839c,87.4,4,21.85,60


In [93]:
def RFMscore(x):
    # recency
    recencyCutoffs = x.DaysSinceLastPurchase.quantile([0.3333, 0.6666])
    x['Recency'] = 2
    x.loc[x['DaysSinceLastPurchase'] < recencyCutoffs.values[0], 'Recency'] = 1
    x.loc[x['DaysSinceLastPurchase'] > recencyCutoffs.values[1], 'Recency'] = 3
    # frequency
    frequencyCutoffs = x.Transaction.quantile([0.3333, 0.6666])
    x['Frequency'] = 2
    x.loc[x['Transaction'] < frequencyCutoffs.values[0], 'Frequency'] = 3
    x.loc[x['Transaction'] > frequencyCutoffs.values[1], 'Frequency'] = 1
    # monetary
    monetaryCutoffs = x.Avg_Transaction_Value.quantile([0.3333, 0.6666])
    x['Monetary'] = 2
    x.loc[x['Avg_Transaction_Value'] < monetaryCutoffs.values[0], 'Monetary'] = 3
    x.loc[x['Avg_Transaction_Value'] > monetaryCutoffs.values[1], 'Monetary'] = 1
    return x

In [94]:
df = RFMscore(all_tranz)

In [96]:
df.head()

,Customers,Sales_Amount,Transaction,Avg_Transaction_Value,DaysSinceLastPurchase,Recency,Frequency,Monetary
0,000644df-aa9b-4880-b551-357288f59223,17.7,2,8.85,50,2,2,2
1,000668dc-83c7-4a1f-92bb-352709cb8294,1389.0,20,69.45,7,1,1,1
2,0006a7d2-246c-4533-97b0-356539d30012,415.2,8,51.90,17,1,1,1
3,000ab955-f4b1-4c74-89bc-3569ecd81562,23.4,3,7.80,4,1,2,3
4,000fe1a3-a44c-4e39-8a70-352b9779839c,87.4,4,21.85,60,2,2,1


In [98]:
df.loc[(df.Recency == 1) & (df.Frequency == 1) & (df.Monetary == 1),'Segment']= 'Best Customers'
df.loc[(df.Recency == 1) & (df.Frequency == 1) & (df.Monetary == 2),'Segment']= 'Loyal Customers'
df.loc[(df.Recency == 1) & (df.Frequency == 1) & (df.Monetary == 3),'Segment']= 'Loyal Customers'
df.loc[(df.Recency == 1) & (df.Frequency == 2) & (df.Monetary == 1),'Segment']= 'Big spenders'
df.loc[(df.Recency == 1) & (df.Frequency == 2) & (df.Monetary == 2),'Segment']= 'Recent low value'
df.loc[(df.Recency == 1) & (df.Frequency == 2) & (df.Monetary == 3),'Segment']= 'Recent low value'
df.loc[(df.Recency == 1) & (df.Frequency == 3) & (df.Monetary == 1),'Segment']= 'Big spenders'
df.loc[(df.Recency == 1) & (df.Frequency == 3) & (df.Monetary == 2),'Segment']= 'Recent low value'
df.loc[(df.Recency == 1) & (df.Frequency == 3) & (df.Monetary == 3),'Segment']= 'Recent low value'
df.loc[(df.Recency == 2) & (df.Frequency == 1) & (df.Monetary == 1),'Segment']= 'Big spenders'
df.loc[(df.Recency == 2) & (df.Frequency == 1) & (df.Monetary == 2),'Segment']= 'Loyal Customers'
df.loc[(df.Recency == 2) & (df.Frequency == 1) & (df.Monetary == 3),'Segment']= 'Loyal Customers'
df.loc[(df.Recency == 2) & (df.Frequency == 2) & (df.Monetary == 1),'Segment']= 'Big spenders'
df.loc[(df.Recency == 2) & (df.Frequency == 2) & (df.Monetary == 2),'Segment']= 'Lappsed'
df.loc[(df.Recency == 2) & (df.Frequency == 2) & (df.Monetary == 3),'Segment']= 'Lappsed'
df.loc[(df.Recency == 2) & (df.Frequency == 3) & (df.Monetary == 1),'Segment']= 'Big spenders'
df.loc[(df.Recency == 2) & (df.Frequency == 3) & (df.Monetary == 2),'Segment']= 'Lappsed'
df.loc[(df.Recency == 2) & (df.Frequency == 3) & (df.Monetary == 3),'Segment']= 'Lappsed'
df.loc[(df.Recency == 3) & (df.Frequency == 1) & (df.Monetary == 1),'Segment']= 'Loyal Lapsed'
df.loc[(df.Recency == 3) & (df.Frequency == 1) & (df.Monetary == 2),'Segment']= 'Loyal Lapsed'
df.loc[(df.Recency == 3) & (df.Frequency == 1) & (df.Monetary == 3),'Segment']= 'Loyal Lapsed'
df.loc[(df.Recency == 3) & (df.Frequency == 2) & (df.Monetary == 1),'Segment']= 'Inactive Accounts'
df.loc[(df.Recency == 3) & (df.Frequency == 2) & (df.Monetary == 2),'Segment']= 'Inactive Accounts'
df.loc[(df.Recency == 3) & (df.Frequency == 2) & (df.Monetary == 3),'Segment']= 'Inactive Accounts'
df.loc[(df.Recency == 3) & (df.Frequency == 3) & (df.Monetary == 1),'Segment']= 'Inactive Accounts'
df.loc[(df.Recency == 3) & (df.Frequency == 3) & (df.Monetary == 2),'Segment']= 'Inactive Accounts'
df.loc[(df.Recency == 3) & (df.Frequency == 3) & (df.Monetary == 3),'Segment']= 'Inactive Accounts'

In [99]:
df.head()

,Customers,Sales_Amount,Transaction,Avg_Transaction_Value,DaysSinceLastPurchase,Recency,Frequency,Monetary,Segment
0,000644df-aa9b-4880-b551-357288f59223,17.7,2,8.85,50,2,2,2,Lappsed
1,000668dc-83c7-4a1f-92bb-352709cb8294,1389.0,20,69.45,7,1,1,1,Best Customers
2,0006a7d2-246c-4533-97b0-356539d30012,415.2,8,51.90,17,1,1,1,Best Customers
3,000ab955-f4b1-4c74-89bc-3569ecd81562,23.4,3,7.80,4,1,2,3,Recent low value
4,000fe1a3-a44c-4e39-8a70-352b9779839c,87.4,4,21.85,60,2,2,1,Big spenders


In [100]:
df.to_csv(os.path.join("..","Data","rfmgreece_output.csv"))